In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 50.2 MB/s eta 0:00:00


In [ ]:
import pulp
from pulp import *

In [ ]:
################################
# Input parameters
###############################

nTime = 10  # number of planning horizons
timeIndex = range(0,nTime)

invInitial = 0  # initial inventory at the beginning
demand = [50, 60, 90, 70, 30, 100, 60, 40, 80, 20]

cap = 9999  # maximum lot size (capacity)
setupCost = 300 # production setup cost
holdingCost = 1 # unit holding cost

In [ ]:
# create model
mod = LpProblem("Lotsizing", LpMinimize)

In [ ]:
################################
# decision variable
###############################

# production quantity in each period
Q = LpVariable.dicts("Qty", timeIndex, lowBound = 0)

# inventory level at end of each period
IL = LpVariable.dicts("Inv", timeIndex, lowBound = 0) # No shortage, so LB=0

# binary for producing or not
U = LpVariable.dicts("UBinary", timeIndex, cat = LpBinary)

In [ ]:
################################
# objective function
###############################
mod += lpSum( (setupCost*U[t] + holdingCost*IL[t]) for t in timeIndex), "TotalCost"

In [ ]:

################################
# constraints
###############################

# constraint: if there is production, then Ubinary must be 1
for t in timeIndex:
    mod += Q[t] <= cap*U[t]

# inventory balance constraint
for t in range(1,nTime):
    mod += IL[t] == IL[t-1] + Q[t] - demand[t]

# inventory balance constraint for 1st period
mod += IL[0] == invInitial + Q[0] - demand[0]

In [ ]:
mod.solve()

1

In [ ]:
print("Status:", LpStatus[mod.status])
print(value(mod.objective))

Status: Optimal
1550.0


In [ ]:
for v in mod.variables():
  if(v.varValue > 0):
    print(v.name, "=", v.varValue)

Inv_0 = 60.0
Inv_2 = 100.0
Inv_3 = 30.0
Inv_5 = 100.0
Inv_6 = 40.0
Inv_8 = 20.0
Qty_0 = 110.0
Qty_2 = 190.0
Qty_5 = 200.0
Qty_8 = 100.0
UBinary_0 = 1.0
UBinary_2 = 1.0
UBinary_5 = 1.0
UBinary_8 = 1.0


In [ ]:
for t in timeIndex:
  if(Q[t].varValue > 0):
    print("Period = ", t+1, ", Order Qyt =", Q[t].varValue)

Period =  1 , Order Qyt = 110.0
Period =  3 , Order Qyt = 190.0
Period =  6 , Order Qyt = 200.0
Period =  9 , Order Qyt = 100.0


In [ ]:
############################################################
# Input parameters
############################################################

nProd = 2   # number of products
prodIndex = range(0, nProd)  # index set of products
nTime = 10  # number of planning horizons
timeIndex = range(0, nTime)  # index set of time periods

# demands of the products
demand = [[50, 60, 90, 70, 30, 100, 60, 40, 80, 20],
          [134, 155, 194, 176, 130, 1100, 158, 140, 193, 100]]

# initial inventories of the products
invInitial = [0, 0]

prodCap = 9999  # maximum lot size of each product
budgetCap = 30000 # budget constraint in each time period

setupCost = [300, 200] # production setup cost
holdingCost = [1.0, 1.5] # unit holding cost
prodCost = [50, 20] # unit production cost


############################################################
# Optimization models
############################################################

mod2 = LpProblem("LotsizingMultiprod", LpMinimize)


# decision variables
Q = LpVariable.dicts("Qty", ((i,t) for i in  prodIndex for t in timeIndex ), lowBound=0)
IL = LpVariable.dicts("Inv", ((i,t) for i in  prodIndex for t in timeIndex ), lowBound=0)
U = LpVariable.dicts("UBinary",  ((i,t) for i in  prodIndex for t in timeIndex ),  cat = LpBinary)


# objective function
mod2 += lpSum((setupCost[i]*U[i,t] + holdingCost[i]*IL[i,t]) for i in prodIndex for t in timeIndex )


# constraint for defining binary variable
for i in prodIndex:
    for t in timeIndex:
        mod2 += Q[i,t] <= prodCap*U[i,t]

# inventory balance constraint
for i in prodIndex:
    for t in range(2,nTime):
        mod2 += IL[i,t] == IL[i,t-1] + Q[i,t] - demand[i][t]

# inventory balance constraint for 1st period
for i in prodIndex:
    mod2 += IL[i,0] == invInitial[i] + Q[i,0] - demand[i][0]


# budget constraint
for t in timeIndex:
    mod2 += lpSum(prodCost[i]*Q[i,t] for i in prodIndex) <= budgetCap

In [ ]:
mod2.solve()

1

In [ ]:
print("Status:", LpStatus[mod2.status])
print(value(mod2.objective))

Status: Optimal
3145.0


In [ ]:
for v in mod2.variables():
  if(v.varValue > 0):
    print(v.name, "=", v.varValue)

Inv_(0,_1) = 190.0
Inv_(0,_2) = 100.0
Inv_(0,_3) = 30.0
Inv_(0,_5) = 60.0
Inv_(0,_7) = 100.0
Inv_(0,_8) = 20.0
Inv_(1,_3) = 130.0
Inv_(1,_8) = 100.0
Qty_(0,_0) = 50.0
Qty_(0,_5) = 160.0
Qty_(0,_7) = 140.0
Qty_(1,_0) = 134.0
Qty_(1,_2) = 194.0
Qty_(1,_3) = 306.0
Qty_(1,_5) = 1100.0
Qty_(1,_6) = 158.0
Qty_(1,_7) = 140.0
Qty_(1,_8) = 293.0
UBinary_(0,_0) = 1.0
UBinary_(0,_5) = 1.0
UBinary_(0,_7) = 1.0
UBinary_(1,_0) = 1.0
UBinary_(1,_2) = 1.0
UBinary_(1,_3) = 1.0
UBinary_(1,_5) = 1.0
UBinary_(1,_6) = 1.0
UBinary_(1,_7) = 1.0
UBinary_(1,_8) = 1.0


In [ ]:
for t in timeIndex:
  if(Q[i,t].varValue > 0):
    print("Period = ", t+1, ", Order Qyt =", Q[i,t].varValue)

Period =  1 , Order Qyt = 134.0
Period =  3 , Order Qyt = 194.0
Period =  4 , Order Qyt = 306.0
Period =  6 , Order Qyt = 1100.0
Period =  7 , Order Qyt = 158.0
Period =  8 , Order Qyt = 140.0
Period =  9 , Order Qyt = 293.0
